In [7]:
!pip install keras_tuner

In [8]:
# Nhập các thư viện cần thiết
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import BayesianOptimization

In [9]:
# Tải và chuẩn bị dữ liệu CIFAR-10
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [10]:
# Định nghĩa hàm xây dựng mô hình CNN
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.InputLayer(input_shape=(32, 32, 3)))

    # Thêm các khối convolutional với số lượng và số lượng bộ lọc được tối ưu hóa bởi BO
    for i in range(hp.Int("conv_blocks", min_value=1, max_value=3, step=1)):
        model.add(layers.Conv2D(filters=hp.Int(f"filters_{i}", min_value=16, max_value=64, step=16),
                                kernel_size=(3, 3), padding="same", activation="relu"))
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    # Thêm một lớp Fully-connected với số lượng nơ-ron được tối ưu hóa bởi BO
    model.add(layers.Flatten())
    model.add(layers.Dense(units=hp.Int("dense_units", min_value=32, max_value=128, step=32),
                           activation="relu"))

    # Thêm một lớp đầu ra với 10 nơ-ron cho 10 lớp của CIFAR-10
    model.add(layers.Dense(10, activation="softmax"))

    # Biên dịch mô hình với tốc độ học được tối ưu hóa bởi BO
    model.compile(optimizer=keras.optimizers.Adam(
        learning_rate=hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log")),
        loss="categorical_crossentropy", metrics=["accuracy"])

    return model

In [11]:
# Khởi tạo một đối tượng BayesianOptimization với hàm xây dựng mô hình và các thông số khác
tuner = BayesianOptimization(build_model,
                             objective="val_accuracy",
                             max_trials=10,
                             executions_per_trial=2,
                             directory="bayesian_optimization",
                             project_name="cnn_cifar10")


In [12]:

from tensorflow.keras.callbacks import CSVLogger

csv_logger = CSVLogger('log.csv', append=True, separator=';')

# Huấn luyện mô hình với callback
tuner.search(x_train, y_train,
             epochs=1,
             validation_data=(x_test, y_test),
             callbacks=[csv_logger])

best_model = tuner.get_best_models(num_models=1)[0]
best_model.evaluate(x_test, y_test)

Trial 5 Complete [00h 01m 12s]
val_accuracy: 0.5456500053405762

Best val_accuracy So Far: 0.5625999867916107
Total elapsed time: 00h 05m 16s

Search: Running Trial #6

Value             |Best Value So Far |Hyperparameter
3                 |2                 |conv_blocks
48                |32                |filters_0
96                |96                |dense_units
0.00032883        |0.00088443        |learning_rate
16                |16                |filters_1
48                |None              |filters_2

 671/1563 [===========>..................] - ETA: 20s - loss: 1.8429 - accuracy: 0.3287

KeyboardInterrupt: 

In [ ]:
import pandas as pd
print(pd.read_csv("training_log.csv"))